In [1]:
# The code was removed by Watson Studio for sharing.

# Capstone Project - The Battle of the Neighborhoods (Week 2)
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>

In my project I will investigate in a general sense similarities betweeen postcodes in Birmingham vs London. The following geospatial analysis will review location data by postcode and aim to form clusters based on location attributes. In this case I will extract data on top venue categories and distinguish which postcodes are similar based on that feature. This could then give insight to a business owner in London to expand on which postcode.

It is known that location is a essential feature for a store or business to perform well. In particular other types of venues could drive footfall to neighbouring venues. Hence a business owner could using clustering to examine areas that are in favourable clusters. I will observe the venues of each post code and and form map visualisations to examine clusters.

The case which I will examine for will be Indian restaurants between both cities and try to recommend which Birmingham postcode a Indian restaurant owner from London would like to consider. Cases of postcodes that we are interested in will be of the following:
* Areas with no Indian restaurants but close to the centre of Birmingham as possible: Opportunity to capitalise on an area missing out
* Areas with Indian restaurants: Established locations with direct rivals may either cannibalise potential sales or let you benefit from footfall that is coming from being near rivals. This idea needs further inspection beyond the scope of this project but potential candidates for this category will be considered

In this notebook I hope to evaluate pros/cons of resulting clusters and give the best recommendation arising from the data

## Data <a name="data"></a>

Firstly we will need to scrape postal location data of Birmingham and London to start with. Then retrieve the coresponding latitude & longittude coordinates of each postcode via geopy. If not then perhaps another website to scrape the information. 

Wikipedia tables will suffice for postcode data scrape. Their postal district tables will be used as a placeholder for a representive/central full postcode ('B1' as opposed to 'B1 1AA'). The Birmingham postal area (first 1-2 characters in a postcode) is just B while London is a much bigger city with 9 different postal districts (e.g. EC for East Central London). 

__Important Notice:__ Nominatim within geopy library will be the API to source geographical coordinates from. It is an open-source API which is volunteer run but have its limits on usage. The method that will be implemented uses geopy but despite some efforts to handle failed requests you may come across a time where geocoding won't work. In practice for regular geocoding, it is advisable to store/cache results or use a premium API for stakeholder needs.

We will then need to make calls to the Foursquare API to request venue data for each location. This will then need some preprocessing to extract venue data from the raw json file into something that can be analysed.

After cleaning the data, we can prepare to run K means clustering and form clusters of post codes in Birmingham/London. The output data will try to label each post code to clusters based on venue categories they offer. We then finally discuss findings from each cluster and how they distinguish from each other.

### Imports

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests # library to handle requests

from bs4 import BeautifulSoup

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import json

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.3 MB

The following NEW packages will be 

### Data scrape for Birmingham postcode district table
Let's request html of the relevant wikipedia page and synthesise the data to have the postcode district table into a pandas dataframe.

In [4]:
bham_wiki_url = requests.get('https://en.wikipedia.org/wiki/B_postcode_area').text # This will ping a website and return the HTML of the website

soup = BeautifulSoup(bham_wiki_url,'lxml') # This creates a BeautifulSoup object via BeautifulSoup function. This package is for parsing HTML and XML documents
#print(soup.prettify()) # 'Prettify' will let us view how the tags are nested in the document
table = soup.find_all('table')[1]
#print(table.prettify())
df1 = pd.read_html(str(table))
df1 = df1[0]
df1.sample(10)

,Postcode district,Post town,Coverage,Local authority area
49,B62,HALESOWEN,"Halesowen (east), Romsley, Hunnington, Quinton...","Dudley, Bromsgrove, Birmingham"
35,B37,BIRMINGHAM,"Chelmsley Wood, Marston Green, Kingshurst, For...",Solihull
46,B50,ALCESTER,Bidford-on-Avon,Stratford-on-Avon
45,B49,ALCESTER,Alcester,Stratford-on-Avon
32,B34,BIRMINGHAM,"Shard End, Buckland End",Birmingham
31,B33,BIRMINGHAM,"Kitts Green, Stechford",Birmingham
19,B20,BIRMINGHAM,"Handsworth Wood, Handsworth, Birchfield",Birmingham
58,B71,WEST BROMWICH,West Bromwich (north),Sandwell
33,B35,BIRMINGHAM,Castle Vale,Birmingham
40,B44,BIRMINGHAM,"Perry Barr, Kingstanding, Great Barr",Birmingham


Now let's clean some of the data and check for null/missing values.

In [5]:
#Here we check the data and try to correct missing values or irrelevant rows. The last row is non-geographical hence we will drop the row
print('Columns in bham dataframe with null values:\n', df1.isnull().sum())
print('-'*10)
print(df1.info())


Columns in bham dataframe with null values:
 Postcode district       0
Post town               0
Coverage                1
Local authority area    0
dtype: int64
----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 4 columns):
Postcode district       78 non-null object
Post town               78 non-null object
Coverage                77 non-null object
Local authority area    78 non-null object
dtypes: object(4)
memory usage: 2.5+ KB
None


We then remove/replace values in preparation for using Geopy.

In [6]:
df1.drop(77, axis=0, inplace=True) #Comment line if already run once
df1.replace('Halesowen (west and town centre), Hayley Green, Hasbury','Halesowen town centre', inplace=True) #Comment line if already run once

print('Columns in bham dataframe with null values:\n', df1.isnull().sum())
print('-'*10)
print(df1.info())

Columns in bham dataframe with null values:
 Postcode district       0
Post town               0
Coverage                0
Local authority area    0
dtype: int64
----------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 77 entries, 0 to 76
Data columns (total 4 columns):
Postcode district       77 non-null object
Post town               77 non-null object
Coverage                77 non-null object
Local authority area    77 non-null object
dtypes: object(4)
memory usage: 3.0+ KB
None


We import geopy and develop a recursive function that aids for exception handling when running geocoder

In [7]:
#!conda install -c conda-forge geopy --yes # comment this line if not needed
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


In [8]:
import geopy.geocoders # import geocoder

In [9]:
from geopy.exc import GeocoderTimedOut
from geopy.exc import GeocoderUnavailable

def do_geocode(address):
    geopy = Nominatim(user_agent="bham_explorer")
    try:
        return geopy.geocode(address, timeout=1)
    except GeocoderUnavailable:
        return do_geocode(address)
    except GeocoderTimedOut:
        return do_geocode(address)

In [10]:
address = 'Smethwick B66, UK'

geolocator = Nominatim(user_agent="bham_explorer")
location = do_geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The geograpical coordinate of Smethwick B66, UK are 52.4968157, -1.9739174.


In [11]:
b96_map = folium.Map(location=[latitude, longitude], zoom_start=15)

b96_map

Here we make final touches to the postcode dataframe for geocoding. We add new feature called 'Address' to be an input column for retrieving coordinates.

In [12]:
df1['Address'] = df1['Coverage'] +', '+ df1['Postcode district']
df1

,Postcode district,Post town,Coverage,Local authority area,Address
0,B1,BIRMINGHAM,"Birmingham City Centre, Broad Street (east)",Birmingham,"Birmingham City Centre, Broad Street (east), B1"
1,B2,BIRMINGHAM,"Birmingham City Centre, New Street",Birmingham,"Birmingham City Centre, New Street, B2"
2,B3,BIRMINGHAM,"Birmingham City Centre, Newhall Street",Birmingham,"Birmingham City Centre, Newhall Street, B3"
3,B4,BIRMINGHAM,"Birmingham City Centre, Corporation Street (no...",Birmingham,"Birmingham City Centre, Corporation Street (no..."
4,B5,BIRMINGHAM,"Digbeth, Highgate, Lee Bank",Birmingham,"Digbeth, Highgate, Lee Bank, B5"
5,B6,BIRMINGHAM,"Aston, Witton",Birmingham,"Aston, Witton, B6"
6,B7,BIRMINGHAM,Nechells,Birmingham,"Nechells, B7"
7,B8,BIRMINGHAM,"Washwood Heath, Ward End, Saltley",Birmingham,"Washwood Heath, Ward End, Saltley, B8"
8,B9,BIRMINGHAM,"Bordesley Green, Bordesley",Birmingham,"Bordesley Green, Bordesley, B9"
9,B10,BIRMINGHAM,Small Heath,Birmingham,"Small Heath, B10"


We create a subset copy to demonstrate geopy in action with roughly 30 postcodes as input.

In [13]:
#df1['location'] = df1['Address'].apply(geolocator.geocode)
df2 = df1.copy(deep=True)
df2 = df2.iloc[:30]
df2.shape

(30, 5)

Let's create a function to apply geopy to all address's and return the cordinates of each postcode

In [15]:
def getLatLngByAddress(postcode, neighbourhood, address):
    
    lat_lng_list=[]
    for pc, nbh, address in zip(postcode, neighbourhood, address):
        print(pc)
        
        #geocoders request for latitude and longitude coordinates
        try:
            print(address)
            location = do_geocode(address)
            latitude = location.latitude
            longitude = location.longitude
        except:
            try:
                print(nbh)
                location = do_geocode(nbh)
                latitude = location.latitude
                longitude = location.longitude
            except:
                try:
                    x = nbh.split(", ")
                    print(x)
                    location = do_geocode(x[0] + ', UK')
                    latitude = location.latitude
                    longitude = location.longitude
                except:
                    try:
                        print(x[1:])
                        location = do_geocode(x[1] + ', UK')
                        latitude = location.latitude
                        longitude = location.longitude
                    except:
                        print(x[2:])
                        location = do_geocode(x[2] + ', UK')
                        latitude = location.latitude
                        longitude = location.longitude

            
        
        lat_lng_list.append([(pc, 
                              nbh, 
                              latitude, 
                              longitude)])
    
    #location_df = pd.DataFrame(data=lat_lng_list)
    #location_df.columns = ['Postcode',
    #                      'Neighbourhood',
    #                      'Latitude',
    #                      'Longitude']
    
    return(lat_lng_list)
        

In [16]:
lat_lng_list = getLatLngByAddress(postcode=df2['Postcode district'], neighbourhood=df2['Coverage'], address=df2['Address'])

B1
Birmingham City Centre, Broad Street (east), B1
Birmingham City Centre, Broad Street (east)
['Birmingham City Centre', 'Broad Street (east)']
B2
Birmingham City Centre, New Street, B2
Birmingham City Centre, New Street
B3
Birmingham City Centre, Newhall Street, B3
Birmingham City Centre, Newhall Street
B4
Birmingham City Centre, Corporation Street (north), B4
Birmingham City Centre, Corporation Street (north)
['Birmingham City Centre', 'Corporation Street (north)']
B5
Digbeth, Highgate, Lee Bank, B5
Digbeth, Highgate, Lee Bank
['Digbeth', 'Highgate', 'Lee Bank']
B6
Aston, Witton, B6
B7
Nechells, B7
B8
Washwood Heath, Ward End, Saltley, B8
B9
Bordesley Green, Bordesley, B9
B10
Small Heath, B10
B11
Sparkhill, Sparkbrook, Tyseley, B11
Sparkhill, Sparkbrook, Tyseley
['Sparkhill', 'Sparkbrook', 'Tyseley']
B12
Balsall Heath, Sparkbrook, Highgate, B12
Balsall Heath, Sparkbrook, Highgate
['Balsall Heath', 'Sparkbrook', 'Highgate']
B13
Moseley, Billesley, B13
B14
Kings Heath, Yardley Wood, D

We tweak the data formated in order to append it to our postcode dataframe.

In [18]:
l1 = np.array(lat_lng_list)
l1.shape
print(np.info(l1))
print('-'*10)
print(l1)

class:  ndarray
shape:  (30, 1, 4)
strides:  (1120, 1120, 280)
itemsize:  280
aligned:  True
contiguous:  True
fortran:  False
data pointer: 0x5593401f3010
byteorder:  little
byteswap:  False
type: <U70
None
----------
[[['B1' 'Birmingham City Centre, Broad Street (east)' '52.4775396'
   '-1.894053']]

 [['B2' 'Birmingham City Centre, New Street' '52.4792602' '-1.8999756']]

 [['B3' 'Birmingham City Centre, Newhall Street' '52.4832071'
   '-1.9054204']]

 [['B4' 'Birmingham City Centre, Corporation Street (north)'
   '52.4775396' '-1.894053']]

 [['B5' 'Digbeth, Highgate, Lee Bank' '52.4734488' '-1.8871192']]

 [['B6' 'Aston, Witton' '52.506768' '-1.8806006']]

 [['B7' 'Nechells' '52.5023095' '-1.8605038']]

 [['B8' 'Washwood Heath, Ward End, Saltley' '52.4865451' '-1.8330255']]

 [['B9' 'Bordesley Green, Bordesley' '52.4778205' '-1.8487612']]

 [['B10' 'Small Heath' '52.4681262' '-1.8513874']]

 [['B11' 'Sparkhill, Sparkbrook, Tyseley' '52.4533095' '-1.8675068']]

 [['B12' 'Balsall He

In [19]:
lat_list = []
lng_list = []
for i in range(30):
    lat_list.append(l1[i][0][2])
    lng_list.append(l1[i][0][3])
    i += 1
    
print('Latitude: \n', lat_list)
print('Longitude: \n', lng_list)
#len(lng_list)

Latitude: 
 ['52.4775396', '52.4792602', '52.4832071', '52.4775396', '52.4734488', '52.506768', '52.5023095', '52.4865451', '52.4778205', '52.4681262', '52.4533095', '52.4572246', '52.4347571', '52.411306', '52.4603261', '52.4805954', '52.45960005', '52.4968772', '52.4996276', '52.5130035', '52.5088327', '52.5232514', '52.5182792', '52.4709562', '52.4616984', '52.4464213', '52.4369951', '52.440222', '52.4273865', '52.4082313']
Longitude: 
 ['-1.894053', '-1.8999756', '-1.9054204', '-1.894053', '-1.8871192', '-1.8806006', '-1.8605038', '-1.8330255', '-1.8487612', '-1.8513874', '-1.8675068', '-1.8832067', '-1.87774549212102', '-1.8657197', '-1.9179098', '-1.918506', '-1.96040213627551', '-1.9380313', '-1.9068745', '-1.9067598', '-1.9387301', '-1.8395109', '-1.8112117', '-1.8159436', '-1.7828480818756', '-1.8210145', '-1.8455973', '-1.9382689', '-1.9266395', '-1.965252']


In [20]:
location_df = pd.DataFrame({ "Latitude" : lat_list, "Longitude" : lng_list})
#df = [df2 , location_df] 
df = df2.join(location_df)
df[['Latitude', 'Longitude']].astype('float64').dtypes


Latitude     float64
Longitude    float64
dtype: object

In [21]:
df.drop(['Post town','Local authority area','Address'], axis=1, inplace=True)

In [22]:
df

,Postcode district,Coverage,Latitude,Longitude
0,B1,"Birmingham City Centre, Broad Street (east)",52.4775396,-1.894053
1,B2,"Birmingham City Centre, New Street",52.4792602,-1.8999756
2,B3,"Birmingham City Centre, Newhall Street",52.4832071,-1.9054204
3,B4,"Birmingham City Centre, Corporation Street (no...",52.4775396,-1.894053
4,B5,"Digbeth, Highgate, Lee Bank",52.4734488,-1.8871192
5,B6,"Aston, Witton",52.506768,-1.8806006
6,B7,Nechells,52.5023095,-1.8605038
7,B8,"Washwood Heath, Ward End, Saltley",52.4865451,-1.8330255
8,B9,"Bordesley Green, Bordesley",52.4778205,-1.8487612
9,B10,Small Heath,52.4681262,-1.8513874


Below you will see a visualisation of the resulting coordinates with markers & labels to each location.

In [23]:
# create map of Birmingham UK using latitude and longitude values
latitude = 52.4862
longitude = -1.8904
map_bham = folium.Map(location=[latitude, longitude], zoom_start=12.3)

# add markers to map
for lat, lng, pc, nbh in zip(df['Latitude'], df['Longitude'], df['Postcode district'], df['Coverage']):
    label = 'Postcode: {}\n, Area: {}\n'.format(pc, nbh)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [float(lat), float(lng)],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bham)  
    
map_bham

### Data scrape for *London postcode district table
Let's request html of the relevant wikipedia page and synthesise the data to have the postcode district table into a pandas dataframe.

One thing to note is that each postcode district reside on separate pages. Therefore we create a funtion to cycle through a list of urls.

We restrict the scope of this study to a sample of postcodes.

In [24]:
postal_district_urls = ['https://en.wikipedia.org/wiki/E_postcode_area', 'https://en.wikipedia.org/wiki/EC_postcode_area', 'https://en.wikipedia.org/wiki/N_postcode_area', 'https://en.wikipedia.org/wiki/WC_postcode_area', 'https://en.wikipedia.org/wiki/W_postcode_area']

In [25]:
def PostalDataframeScrapeLDN(urls):
    df_list = []
    for i in range(len(urls)):
        ldn_wiki_url = requests.get(urls[i]).text
        soup = BeautifulSoup(ldn_wiki_url,'lxml') # This creates a BeautifulSoup object via BeautifulSoup function. This package is for parsing HTML and XML documents
        table = soup.find_all('table')[1]
        df1 = pd.read_html(str(table))
        df1 = df1[0]
        df_list.append(df1)
    
    df = pd.concat(df_list).reset_index(drop=True)
    return df
        

In [26]:
df_ldn = PostalDataframeScrapeLDN(urls=postal_district_urls)

In [27]:
df_ldn

,Postcode district,Post town,Coverage,Local authority area
0,E1,LONDON,"Eastern head district: Aldgate (part), Bishops...","Tower Hamlets, Hackney, City of London"
1,E1W,LONDON,"Wapping, St Katharine Docks, Stepney (part), S...",Tower Hamlets
2,E2,LONDON,"Bethnal Green district: Bethnal Green, Haggers...","Tower Hamlets, Hackney"
3,E3,LONDON,"Bow district: Bow, Bow Common, Bromley-by-Bow,...","Tower Hamlets, Newham"
4,E4,LONDON,"Chingford district: Chingford, Sewardstone, Hi...","Waltham Forest, Enfield, Epping Forest (Essex)"
5,E5,LONDON,"Clapton district: Leyton (Part), Upper Clapton...","Hackney, Waltham Forest"
6,E6,LONDON,"East Ham district: East Ham, Beckton, Upton Pa...","Newham, Barking and Dagenham"
7,E7,LONDON,"Forest Gate district: Forest Gate, Leytonstone...","Newham, Waltham Forest"
8,E8,LONDON,"Hackney district: Hackney Central, Dalston, Lo...",Hackney
9,E9,LONDON,"Homerton district: Homerton, Hackney Wick, Sou...","Hackney, Tower Hamlets"


Let's proceed with some data corrections.

In [28]:
df_ldn.shape

(115, 4)

In [29]:
#Here we check the data and try to correct missing values or irrelevant rows. The last row is non-geographical hence we will drop the row
print('Columns in ldn dataframe with null values:\n', df_ldn.isnull().sum())
print('-'*10)
print(df_ldn.info())



Columns in ldn dataframe with null values:
 Postcode district       0
Post town               0
Coverage                5
Local authority area    0
dtype: int64
----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 4 columns):
Postcode district       115 non-null object
Post town               115 non-null object
Coverage                110 non-null object
Local authority area    115 non-null object
dtypes: object(4)
memory usage: 3.7+ KB
None


In [32]:
#df_ldn = df_ldn.loc[df_ldn['Local authority area'] != 'non-geographic', :].reset_index(drop=True) #Comment line if already run once
#df_ldn.drop(index=[20,21], inplace=True)
#df_ldn.reset_index(drop=True, inplace=True)

print('Columns in ldn dataframe with null values:\n', df_ldn.isnull().sum())
print('-'*10)
print(df_ldn.info())

Columns in ldn dataframe with null values:
 Postcode district       0
Post town               0
Coverage                0
Local authority area    0
dtype: int64
----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 4 columns):
Postcode district       105 non-null object
Post town               105 non-null object
Coverage                105 non-null object
Local authority area    105 non-null object
dtypes: object(4)
memory usage: 3.4+ KB
None


In [33]:
df_ldn

,Postcode district,Post town,Coverage,Local authority area
0,E1,LONDON,"Eastern head district: Aldgate (part), Bishops...","Tower Hamlets, Hackney, City of London"
1,E1W,LONDON,"Wapping, St Katharine Docks, Stepney (part), S...",Tower Hamlets
2,E2,LONDON,"Bethnal Green district: Bethnal Green, Haggers...","Tower Hamlets, Hackney"
3,E3,LONDON,"Bow district: Bow, Bow Common, Bromley-by-Bow,...","Tower Hamlets, Newham"
4,E4,LONDON,"Chingford district: Chingford, Sewardstone, Hi...","Waltham Forest, Enfield, Epping Forest (Essex)"
5,E5,LONDON,"Clapton district: Leyton (Part), Upper Clapton...","Hackney, Waltham Forest"
6,E6,LONDON,"East Ham district: East Ham, Beckton, Upton Pa...","Newham, Barking and Dagenham"
7,E7,LONDON,"Forest Gate district: Forest Gate, Leytonstone...","Newham, Waltham Forest"
8,E8,LONDON,"Hackney district: Hackney Central, Dalston, Lo...",Hackney
9,E9,LONDON,"Homerton district: Homerton, Hackney Wick, Sou...","Hackney, Tower Hamlets"


We create the 'Address' variable and prepare to use geopy onto it. Below we removed rows that returned no coordinates as more complete data is needed for Nominatim to return coordinates.

In [35]:
df_ldn['Address'] = df_ldn['Coverage'] +', '+ df_ldn['Postcode district']
df2 = df_ldn.copy(deep=True)
df2 = df2.iloc[:30]
df2.reset_index(drop=True, inplace=True)

In [40]:
df2.drop([3, 7, 12, 14, 15, 18, 19, 28, 29], axis=0, inplace=True)
df2

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,Postcode district,Post town,Coverage,Local authority area,Address
0,E1,LONDON,"Eastern head district: Aldgate (part), Bishops...","Tower Hamlets, Hackney, City of London","Eastern head district: Aldgate (part), Bishops..."
1,E1W,LONDON,"Wapping, St Katharine Docks, Stepney (part), S...",Tower Hamlets,"Wapping, St Katharine Docks, Stepney (part), S..."
2,E2,LONDON,"Bethnal Green district: Bethnal Green, Haggers...","Tower Hamlets, Hackney","Bethnal Green district: Bethnal Green, Haggers..."
4,E4,LONDON,"Chingford district: Chingford, Sewardstone, Hi...","Waltham Forest, Enfield, Epping Forest (Essex)","Chingford district: Chingford, Sewardstone, Hi..."
5,E5,LONDON,"Clapton district: Leyton (Part), Upper Clapton...","Hackney, Waltham Forest","Clapton district: Leyton (Part), Upper Clapton..."
6,E6,LONDON,"East Ham district: East Ham, Beckton, Upton Pa...","Newham, Barking and Dagenham","East Ham district: East Ham, Beckton, Upton Pa..."
8,E8,LONDON,"Hackney district: Hackney Central, Dalston, Lo...",Hackney,"Hackney district: Hackney Central, Dalston, Lo..."
9,E9,LONDON,"Homerton district: Homerton, Hackney Wick, Sou...","Hackney, Tower Hamlets","Homerton district: Homerton, Hackney Wick, Sou..."
10,E10,LONDON,"Leyton district: Leyton, Temple Mills, Hackney...","Waltham Forest, Hackney","Leyton district: Leyton, Temple Mills, Hackney..."
11,E11,LONDON,"Leytonstone district: Leytonstone, Wanstead, A...","Waltham Forest, Redbridge","Leytonstone district: Leytonstone, Wanstead, A..."


We then apply Geopy and get the results appended to our london dataframe.

In [41]:
lat_lng_list = getLatLngByAddress(postcode=df2['Postcode district'], neighbourhood=df2['Coverage'], address=df2['Address'])

E1
Eastern head district: Aldgate (part), Bishopsgate (part), Whitechapel, Shoreditch (part), Spitalfields, Shadwell, Stepney, Mile End (part), Portsoken, E1
Eastern head district: Aldgate (part), Bishopsgate (part), Whitechapel, Shoreditch (part), Spitalfields, Shadwell, Stepney, Mile End (part), Portsoken
['Eastern head district: Aldgate (part)', 'Bishopsgate (part)', 'Whitechapel', 'Shoreditch (part)', 'Spitalfields', 'Shadwell', 'Stepney', 'Mile End (part)', 'Portsoken']
['Bishopsgate (part)', 'Whitechapel', 'Shoreditch (part)', 'Spitalfields', 'Shadwell', 'Stepney', 'Mile End (part)', 'Portsoken']
['Whitechapel', 'Shoreditch (part)', 'Spitalfields', 'Shadwell', 'Stepney', 'Mile End (part)', 'Portsoken']
E1W
Wapping, St Katharine Docks, Stepney (part), Shadwell (part), Whitechapel (part), E1W
Wapping, St Katharine Docks, Stepney (part), Shadwell (part), Whitechapel (part)
['Wapping', 'St Katharine Docks', 'Stepney (part)', 'Shadwell (part)', 'Whitechapel (part)']
E2
Bethnal Green d

In [43]:
l1 = np.array(lat_lng_list)
l1.shape
print(np.info(l1))
print('-'*25)
print(l1)


class:  ndarray
shape:  (21, 1, 4)
strides:  (2400, 2400, 600)
itemsize:  600
aligned:  True
contiguous:  True
fortran:  False
data pointer: 0x5593406c6910
byteorder:  little
byteswap:  False
type: <U150
None
-------------------------
[[['E1'
   'Eastern head district: Aldgate (part), Bishopsgate (part), Whitechapel, Shoreditch (part), Spitalfields, Shadwell, Stepney, Mile End (part), Portsoken'
   '51.5195786' '-0.0606907']]

 [['E1W'
   'Wapping, St Katharine Docks, Stepney (part), Shadwell (part), Whitechapel (part)'
   '51.5054363' '-0.0587291']]

 [['E2'
   'Bethnal Green district: Bethnal Green, Haggerston, Hoxton (part), Shoreditch (part), Cambridge Heath, Globe Town, Shoreditch (part)'
   '51.5385184' '-0.0753811']]

 [['E4'
   'Chingford district: Chingford, Sewardstone, Highams Park, Upper Edmonton (part), Woodford Green (part)'
   '51.6245825' '-0.0096957']]

 [['E5'
   'Clapton district: Leyton (Part), Upper Clapton, Lower Clapton, Stoke Newington (part)'
   '51.5668384' '-

In [45]:
lat_list = []
lng_list = []
for i in range(21):
    lat_list.append(l1[i][0][2])
    lng_list.append(l1[i][0][3])
    i += 1
    
print(lat_list)
print(lng_list)

['51.5195786', '51.5054363', '51.5385184', '51.6245825', '51.5668384', '51.5160797', '51.5434023', '51.5434262', '51.8146817', '51.5756739', '51.5310993', '51.5013626', '51.5879893', '51.5174843', '51.5217955', '51.5205785', '51.52649225', '51.5265742', '51.52366865', '51.5220564', '51.5183579']
['-0.0606907', '-0.0587291', '-0.0753811', '-0.0096957', '-0.0605697', '0.0594257', '-0.0672286', '-0.0277034', '-0.356914287847605', '0.0277989', '0.016283', '0.0385185', '0.0020978', '-0.1012598', '-0.1066298', '-0.1085147', '-0.106115781001953', '-0.1070713', '-0.0887493055891544', '-0.081771', '-0.0821607']


In [51]:
#location_df = pd.DataFrame({ "Latitude" : lat_list, "Longitude" : lng_list})
#df2.reset_index(drop=True, inplace=True) 
df_ldn = df2.join(location_df)
df_ldn[['Latitude', 'Longitude']].astype('float64').dtypes
df_ldn.reset_index(drop=True, inplace=True)
df_ldn
#location_df


,Postcode district,Post town,Coverage,Local authority area,Address,Latitude,Longitude
0,E1,LONDON,"Eastern head district: Aldgate (part), Bishops...","Tower Hamlets, Hackney, City of London","Eastern head district: Aldgate (part), Bishops...",51.5195786,-0.0606907
1,E1W,LONDON,"Wapping, St Katharine Docks, Stepney (part), S...",Tower Hamlets,"Wapping, St Katharine Docks, Stepney (part), S...",51.5054363,-0.0587291
2,E2,LONDON,"Bethnal Green district: Bethnal Green, Haggers...","Tower Hamlets, Hackney","Bethnal Green district: Bethnal Green, Haggers...",51.5385184,-0.0753811
3,E4,LONDON,"Chingford district: Chingford, Sewardstone, Hi...","Waltham Forest, Enfield, Epping Forest (Essex)","Chingford district: Chingford, Sewardstone, Hi...",51.6245825,-0.0096957
4,E5,LONDON,"Clapton district: Leyton (Part), Upper Clapton...","Hackney, Waltham Forest","Clapton district: Leyton (Part), Upper Clapton...",51.5668384,-0.0605697
5,E6,LONDON,"East Ham district: East Ham, Beckton, Upton Pa...","Newham, Barking and Dagenham","East Ham district: East Ham, Beckton, Upton Pa...",51.5160797,0.0594257
6,E8,LONDON,"Hackney district: Hackney Central, Dalston, Lo...",Hackney,"Hackney district: Hackney Central, Dalston, Lo...",51.5434023,-0.0672286
7,E9,LONDON,"Homerton district: Homerton, Hackney Wick, Sou...","Hackney, Tower Hamlets","Homerton district: Homerton, Hackney Wick, Sou...",51.5434262,-0.0277034
8,E10,LONDON,"Leyton district: Leyton, Temple Mills, Hackney...","Waltham Forest, Hackney","Leyton district: Leyton, Temple Mills, Hackney...",51.8146817,-0.356914287847605
9,E11,LONDON,"Leytonstone district: Leytonstone, Wanstead, A...","Waltham Forest, Redbridge","Leytonstone district: Leytonstone, Wanstead, A...",51.5756739,0.0277989


Let's inspect for anomalous coordinates and remove them from the dataset. This can be quickly visualised on the folium map code below.

In [160]:
len(lat_list)

28

In [169]:
#df_ldn.drop([3, 7, 12, 14, 15, 18, 19, 28, 29], axis=0, inplace=True)
df_ldn.drop([10], axis=0, inplace=True)

In [52]:
# create map of London UK using latitude and longitude values
latitude = 51.5074
longitude = -0.1278
map_ldn = folium.Map(location=[latitude, longitude], zoom_start=12.3)

# add markers to map
for lat, lng, pc, nbh in zip(df_ldn['Latitude'], df_ldn['Longitude'], df_ldn['Postcode district'], df_ldn['Coverage']):
    label = 'Postcode: {}\n, Area: {}\n'.format(pc, nbh)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [float(lat), float(lng)],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ldn)  
    
map_ldn

Final touches to the postcode location data from London before merging with the Birmingham table. We remove redundant features for simplicity.

In [53]:
df_ldn.drop(['Post town','Local authority area','Address'], axis=1, inplace=True)

In [54]:
df_ldn

,Postcode district,Coverage,Latitude,Longitude
0,E1,"Eastern head district: Aldgate (part), Bishops...",51.5195786,-0.0606907
1,E1W,"Wapping, St Katharine Docks, Stepney (part), S...",51.5054363,-0.0587291
2,E2,"Bethnal Green district: Bethnal Green, Haggers...",51.5385184,-0.0753811
3,E4,"Chingford district: Chingford, Sewardstone, Hi...",51.6245825,-0.0096957
4,E5,"Clapton district: Leyton (Part), Upper Clapton...",51.5668384,-0.0605697
5,E6,"East Ham district: East Ham, Beckton, Upton Pa...",51.5160797,0.0594257
6,E8,"Hackney district: Hackney Central, Dalston, Lo...",51.5434023,-0.0672286
7,E9,"Homerton district: Homerton, Hackney Wick, Sou...",51.5434262,-0.0277034
8,E10,"Leyton district: Leyton, Temple Mills, Hackney...",51.8146817,-0.356914287847605
9,E11,"Leytonstone district: Leytonstone, Wanstead, A...",51.5756739,0.0277989


In [121]:
df_bham = df.drop(['Post town','Local authority area','Address'], axis=1, inplace=True)

In [55]:
df_bham = df.copy(deep=True)
df_bham

,Postcode district,Coverage,Latitude,Longitude
0,B1,"Birmingham City Centre, Broad Street (east)",52.4775396,-1.894053
1,B2,"Birmingham City Centre, New Street",52.4792602,-1.8999756
2,B3,"Birmingham City Centre, Newhall Street",52.4832071,-1.9054204
3,B4,"Birmingham City Centre, Corporation Street (no...",52.4775396,-1.894053
4,B5,"Digbeth, Highgate, Lee Bank",52.4734488,-1.8871192
5,B6,"Aston, Witton",52.506768,-1.8806006
6,B7,Nechells,52.5023095,-1.8605038
7,B8,"Washwood Heath, Ward End, Saltley",52.4865451,-1.8330255
8,B9,"Bordesley Green, Bordesley",52.4778205,-1.8487612
9,B10,Small Heath,52.4681262,-1.8513874


In [56]:
df = pd.concat([df_bham, df_ldn]).reset_index(drop=True)

In [57]:
df

,Postcode district,Coverage,Latitude,Longitude
0,B1,"Birmingham City Centre, Broad Street (east)",52.4775396,-1.894053
1,B2,"Birmingham City Centre, New Street",52.4792602,-1.8999756
2,B3,"Birmingham City Centre, Newhall Street",52.4832071,-1.9054204
3,B4,"Birmingham City Centre, Corporation Street (no...",52.4775396,-1.894053
4,B5,"Digbeth, Highgate, Lee Bank",52.4734488,-1.8871192
5,B6,"Aston, Witton",52.506768,-1.8806006
6,B7,Nechells,52.5023095,-1.8605038
7,B8,"Washwood Heath, Ward End, Saltley",52.4865451,-1.8330255
8,B9,"Bordesley Green, Bordesley",52.4778205,-1.8487612
9,B10,Small Heath,52.4681262,-1.8513874


The location data is now set and ready to use in conjunction to the Foursquare API.

### Foursquare
Now that we have our locations of interest, let's use Foursquare API to get info on venues in each postcode.

Although most categories of venues will not be of interest, it is worth knowing what sort of ammenities are present to gauge where an Indian restaurant could be set up.

We look at venues within 1km radius from each postcode's lat/long pair. We cap the # of venues returned to be at most 50.

In [58]:
# The code was removed by Watson Studio for sharing.

In [59]:
radius = 1000
LIMIT = 50

The function below retrieves nearest venues and some basic info of the venue (such as venue name, category).

In [60]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Postcode', 
                  'Postcode Latitude', 
                  'Postcode Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [61]:
total_venues = getNearbyVenues(names=df['Postcode district'], 
                                 latitudes=df['Latitude'], 
                                 longitudes=df['Longitude']
                                )

B1
B2
B3
B4
B5
B6
B7
B8
B9
B10
B11
B12
B13
B14
B15
B16
B17
B18
B19
B20
B21
B23
B24
B25
B26
B27
B28
B29
B30
B31
E1
E1W
E2
E4
E5
E6
E8
E9
E10
E11
E13
E16
E17
EC1A
EC1M
EC1N
EC1R
EC1V
EC1Y
EC2A
EC2M


In [62]:
print(total_venues.shape)
total_venues.sample(15)

(1065, 7)


,Postcode,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
115,B3,52.4832071,-1.9054204,Birmingham Town Hall,52.479509,-1.903560,Concert Hall
587,E10,51.8146817,-0.356914287847605,Cross Keys,51.816618,-0.356813,Bar
564,E9,51.5434262,-0.0277034,Barge East,51.542177,-0.021046,English Restaurant
90,B2,52.4792602,-1.8999756,The Lost & Found,52.480122,-1.900658,Cocktail Bar
945,EC1Y,51.52366865,-0.0887493055891544,Barbican Cinemas 2&3,51.521073,-0.092979,Indie Movie Theater
586,E10,51.8146817,-0.356914287847605,Prezzo,51.814269,-0.356182,Italian Restaurant
125,B3,52.4832071,-1.9054204,The Old Joint Stock Theatre,52.480825,-1.899973,Pub
931,EC1Y,51.52366865,-0.0887493055891544,Sunny's Olive Tree,51.523177,-0.093004,Food Truck
905,EC1V,51.5265742,-0.1070713,Jimmy And The Bee,51.526402,-0.100223,Coffee Shop
447,E1W,51.5054363,-0.0587291,Tobacco Dock,51.508642,-0.059542,Event Space


## Methodology <a name="methodology"></a>

Now that we have used Foursquare we are prepared for the next step. We have acquired our dataset which gives us venues within 1km radius in our selection of postcodes.

We will need to do some exploratory data analysis and to proceed with preparing our input for clustering. In this project we have allowed all types of categories so that the stakeholder can consider the venue landscape on a location and decide if certain ammenities could be great to be near to. The dataframe we have will turn into a table containing fractions of existing venue categories in each post code. This is done alongside some EDA.

The last part of analysis will be using the sklearn KMeans algorithm to segment postcodes into 6 clusters. The results are then visualised and we discuss the results of each cluster. Each cluster should be able to offer a distinction from each other and I will then recommend area's to consider.



## Analysis <a name="analysis"></a>

Let's observe some summary stats before producing dummy variables to analyse categorical data.

In [63]:
total_venues.groupby('Postcode').count()


,Postcode Latitude,Postcode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Postcode,,,,,,
B1,50,50,50,50,50,50
B10,6,6,6,6,6,6
B11,4,4,4,4,4,4
B12,14,14,14,14,14,14
B13,3,3,3,3,3,3
B14,4,4,4,4,4,4
B15,5,5,5,5,5,5
B16,10,10,10,10,10,10
B17,7,7,7,7,7,7


In [64]:
print('There are {} uniques categories.'.format(len(total_venues['Venue Category'].unique())))

There are 198 uniques categories.


In [65]:
# one hot encoding: creating columns for each unique category and using binary indicator variable for each row from toronto venues dataframe
total_onehot = pd.get_dummies(total_venues[['Venue Category']], prefix="", prefix_sep="")

# add postcode column back to dataframe
total_onehot['Postcode'] = total_venues['Postcode'] 

# move postcode column to the first column
fixed_columns = [total_onehot.columns[-1]] + list(total_onehot.columns[:-1])
total_onehot = total_onehot[fixed_columns]

total_onehot.head()

,Postcode,Afghan Restaurant,Airport Terminal,American Restaurant,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Studio,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Bakery,Bar,Beer Bar,Betting Shop,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Bus Stop,Café,Canal,Candy Store,Cantonese Restaurant,Casino,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Colombian Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cricket Ground,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Discount Store,Distillery,Dive Bar,Doner Restaurant,Donut Shop,Electronics Store,English Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food,Food & Drink Shop,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,History Museum,Home Service,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Lake,Laser Tag,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Moving Target,Multiplex,Museum,Music Venue,Nightclub,Noodle House,North Indian Restaurant,Organic Grocery,Other Repair Shop,Outdoor Sculpture,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Post Office,Pub,Rafting,Ramen Restaurant,Record Shop,Recreation Center,Restaurant,River,Roof Deck,Rugby Pitch,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Shopping Mall,Shopping Plaza,Skating Rink,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Souvlaki Shop,Spanish Restaurant,Speakeasy,Sports Bar,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Track Stadium,Trail,Train Station,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,B1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,B1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,B1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [66]:
total_onehot.shape

(1065, 199)

Now we aggregate the data by postcode districts by taking the mean value for each group's column.

In [67]:
total_grouped = total_onehot.groupby('Postcode').mean().reset_index()
total_grouped

,Postcode,Afghan Restaurant,Airport Terminal,American Restaurant,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Studio,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Auto Garage,Automotive Shop,BBQ Joint,Baby Store,Bakery,Bar,Beer Bar,Betting Shop,Bike Shop,Bistro,Bookstore,Boutique,Bowling Alley,Boxing Gym,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Bus Stop,Café,Canal,Candy Store,Cantonese Restaurant,Casino,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Colombian Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cricket Ground,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Discount Store,Distillery,Dive Bar,Doner Restaurant,Donut Shop,Electronics Store,English Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food,Food & Drink Shop,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,History Museum,Home Service,Hostel,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Lake,Laser Tag,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Moving Target,Multiplex,Museum,Music Venue,Nightclub,Noodle House,North Indian Restaurant,Organic Grocery,Other Repair Shop,Outdoor Sculpture,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Post Office,Pub,Rafting,Ramen Restaurant,Record Shop,Recreation Center,Restaurant,River,Roof Deck,Rugby Pitch,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Shopping Mall,Shopping Plaza,Skating Rink,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Souvlaki Shop,Spanish Restaurant,Speakeasy,Sports Bar,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Track Stadium,Trail,Train Station,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,B1,0.000000,0.0,0.000000,0.0,0.000000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.020000,0.0,0.000000,0.000000,0.000000,0.000000,0.060000,0.000000,0.0,0.00,0.000000,0.040000,0.02,0.000000,0.00,0.000000,0.000000,0.080000,0.00,0.000000,0.000000,0.040000,0.000000,0.000000,0.000000,0.00,0.020000,0.00,0.000000,0.080000,0.000000,0.040000,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.04,0.000000,0.000000,0.00,0.00000,0.02,0.02,0.020000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.020000,0.02,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.02000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.020000,0.000000,0.000000,0.020000,0.020000,0.04,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.02,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.02,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.00000,0.00,0.00,0.020000,0.000000,0.000000,0.00000,0.000000,0.040000,0.000000,0.020000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0

In [68]:
total_grouped.shape

(51, 199)

Let's create a view of top 5 venue categories:

In [69]:
num_top_venues = 5

for pc in total_grouped['Postcode']:
    print("----"+pc+"----")
    temp = total_grouped[total_grouped['Postcode'] == pc].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----B1----
                   venue  freq
0         Clothing Store  0.08
1           Burger Joint  0.08
2                    Bar  0.06
3    Japanese Restaurant  0.04
4  Portuguese Restaurant  0.04


----B10----
                venue  freq
0      Sandwich Place  0.33
1   Indian Restaurant  0.17
2                Park  0.17
3                Café  0.17
4  Turkish Restaurant  0.17


----B11----
                  venue  freq
0  Pakistani Restaurant  0.50
1     Indian Restaurant  0.25
2     Convenience Store  0.25
3     Afghan Restaurant  0.00
4              Pie Shop  0.00


----B12----
                  venue  freq
0     Indian Restaurant  0.36
1  Fast Food Restaurant  0.14
2                   Pub  0.07
3  Pakistani Restaurant  0.07
4                  Pool  0.07


----B13----
                venue  freq
0                 Pub  0.33
1       Moving Target  0.33
2         Golf Course  0.33
3   Afghan Restaurant  0.00
4  Photography Studio  0.00


----B14----
               venue  freq
0      Gro

In [71]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Postcode']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
postcode_venues_sorted = pd.DataFrame(columns=columns)
postcode_venues_sorted['Postcode'] = total_grouped['Postcode']

for ind in np.arange(total_grouped.shape[0]):
    postcode_venues_sorted.iloc[ind, 1:] = return_most_common_venues(total_grouped.iloc[ind, :], num_top_venues)

postcode_venues_sorted

,Postcode,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,B1,Burger Joint,Clothing Store,Bar,Portuguese Restaurant,Bookstore
1,B10,Sandwich Place,Indian Restaurant,Park,Café,Turkish Restaurant
2,B11,Pakistani Restaurant,Indian Restaurant,Convenience Store,Electronics Store,Food & Drink Shop
3,B12,Indian Restaurant,Fast Food Restaurant,Grocery Store,Pool,Café
4,B13,Pub,Moving Target,Golf Course,Yoga Studio,Electronics Store
5,B14,Convenience Store,Gas Station,Grocery Store,Pharmacy,Yoga Studio
6,B15,Tennis Court,Lake,Tennis Stadium,Golf Course,Yoga Studio
7,B16,Italian Restaurant,Restaurant,Café,Bar,Theater
8,B17,Cantonese Restaurant,Deli / Bodega,Coffee Shop,Gym / Fitness Center,Thai Restaurant
9,B18,Light Rail Station,Vegetarian / Vegan Restaurant,Pizza Place,Yoga Studio,Electronics Store


### Clustering Postcodes
We will run K-means to cluster the postcodes into 6 clusters. Firstly we set up some parameters necessary for the algorithm

In [108]:
# set number of clusters
kclusters = 6

total_grouped_clustering = total_grouped.drop('Postcode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(total_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 3, 1, 0, 1, 1, 1, 1], dtype=int32)

In [109]:
# add clustering labels
#postcode_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
postcode_venues_sorted['Cluster Labels'] = kmeans.labels_

total_merged = df

# merge total_grouped with total_data to add latitude/longitude for each neighborhood
total_merged = total_merged.join(postcode_venues_sorted.set_index('Postcode'), on='Postcode district')
total_merged = total_merged.dropna()
total_merged['Cluster Labels'] = total_merged['Cluster Labels'].astype('int64')
total_merged.head() # check the last columns!

,Postcode district,Coverage,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,B1,"Birmingham City Centre, Broad Street (east)",52.4775396,-1.894053,1,Burger Joint,Clothing Store,Bar,Portuguese Restaurant,Bookstore
1,B2,"Birmingham City Centre, New Street",52.4792602,-1.8999756,1,Coffee Shop,Pub,Bar,Italian Restaurant,Bistro
2,B3,"Birmingham City Centre, Newhall Street",52.4832071,-1.9054204,1,Pub,Indian Restaurant,Bar,Coffee Shop,Cocktail Bar
3,B4,"Birmingham City Centre, Corporation Street (no...",52.4775396,-1.894053,1,Burger Joint,Clothing Store,Bar,Portuguese Restaurant,Bookstore
4,B5,"Digbeth, Highgate, Lee Bank",52.4734488,-1.8871192,1,Pub,Music Venue,Bar,Café,Indian Restaurant


In [110]:
total_merged.shape

(51, 10)

In [111]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(total_merged['Latitude'], total_merged['Longitude'], total_merged['Postcode district'], total_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [float(lat), float(lon)],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [120]:
map_clusters

In [112]:
total_merged.groupby('Cluster Labels').count()

,Postcode district,Coverage,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
Cluster Labels,,,,,,,,,
0,3,3,3,3,3,3,3,3,3
1,40,40,40,40,40,40,40,40,40
2,1,1,1,1,1,1,1,1,1
3,4,4,4,4,4,4,4,4,4
4,2,2,2,2,2,2,2,2,2
5,1,1,1,1,1,1,1,1,1


#### Cluster 1 - "Groceries Store" dominant category with odd few venues. Significantly <50 venues based on search criteria used

In [115]:
total_merged.loc[total_merged['Cluster Labels'] == 0, total_merged.columns[[0,1] + list(range(5, total_merged.shape[1]))]]

,Postcode district,Coverage,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
13,B14,"Kings Heath, Yardley Wood, Druids Heath, Hight...",Convenience Store,Gas Station,Grocery Store,Pharmacy,Yoga Studio
21,B23,"Erdington, Stockland Green, Short Heath, Perry...",Grocery Store,Pub,Discount Store,Betting Shop,Clothing Store
29,B31,"Northfield, Longbridge, West Heath",Grocery Store,Pub,Train Station,Donut Shop,Food & Drink Shop


#### Cluster 2 - Typical high streets, town centres with bars and variety of restaurants too. This cluster was the only cluster to return at most 50 venues for a postcode.

In [116]:
total_merged.loc[total_merged['Cluster Labels'] == 1, total_merged.columns[[0,1] + list(range(5, total_merged.shape[1]))]]

,Postcode district,Coverage,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,B1,"Birmingham City Centre, Broad Street (east)",Burger Joint,Clothing Store,Bar,Portuguese Restaurant,Bookstore
1,B2,"Birmingham City Centre, New Street",Coffee Shop,Pub,Bar,Italian Restaurant,Bistro
2,B3,"Birmingham City Centre, Newhall Street",Pub,Indian Restaurant,Bar,Coffee Shop,Cocktail Bar
3,B4,"Birmingham City Centre, Corporation Street (no...",Burger Joint,Clothing Store,Bar,Portuguese Restaurant,Bookstore
4,B5,"Digbeth, Highgate, Lee Bank",Pub,Music Venue,Bar,Café,Indian Restaurant
6,B7,Nechells,Fast Food Restaurant,Italian Restaurant,Bowling Alley,Indian Restaurant,Laser Tag
7,B8,"Washwood Heath, Ward End, Saltley",Bar,Auto Garage,Home Service,Asian Restaurant,English Restaurant
9,B10,Small Heath,Sandwich Place,Indian Restaurant,Park,Café,Turkish Restaurant
10,B11,"Sparkhill, Sparkbrook, Tyseley",Pakistani Restaurant,Indian Restaurant,Convenience Store,Electronics Store,Food & Drink Shop
12,B13,"Moseley, Billesley",Pub,Moving Target,Golf Course,Yoga Studio,Electronics Store


#### Cluster 4 - 'Indian Restaurant' dominant (with the exception of B6). Elements of football/soccer.

In [117]:
total_merged.loc[total_merged['Cluster Labels'] == 3, total_merged.columns[[0,1] + list(range(5, total_merged.shape[1]))]]

,Postcode district,Coverage,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
5,B6,"Aston, Witton",Soccer Stadium,Pub,Park,Soccer Field,Food & Drink Shop
11,B12,"Balsall Heath, Sparkbrook, Highgate",Indian Restaurant,Fast Food Restaurant,Grocery Store,Pool,Café
18,B19,"Lozells, Newtown, Birchfield",Indian Restaurant,Fast Food Restaurant,Bus Stop,Soccer Stadium,Soccer Field
22,B24,"Erdington, Wylde Green, Tyburn",Fast Food Restaurant,Indian Restaurant,American Restaurant,Furniture / Home Store,Yoga Studio


#### Cluster 5 - Parks & Natural reserves with nearby Pubs. Significantly <50 venues per postcode.

In [118]:
total_merged.loc[total_merged['Cluster Labels'] == 4, total_merged.columns[[0,1] + list(range(5, total_merged.shape[1]))]]

,Postcode district,Coverage,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
24,B26,"Sheldon, Yardley",Pub,Park,Yoga Studio,Electronics Store,Food & Drink Shop
33,E4,"Chingford district: Chingford, Sewardstone, Hi...",Park,Pub,Yoga Studio,Electronics Store,Food & Drink Shop


#### Cluster 6 (and 3) - Outliers

In [119]:
total_merged.loc[total_merged['Cluster Labels'] == 5, total_merged.columns[[0,1] + list(range(5, total_merged.shape[1]))]]

,Postcode district,Coverage,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
19,B20,"Handsworth Wood, Handsworth, Birchfield",Discount Store,Train Station,Fast Food Restaurant,Donut Shop,Food & Drink Shop


## Results and Discussion <a name="results"></a>

The analysis segments each postcode to 6 clusters that try to distinguish them apart. This showed 40 out of 51 postcodes used were likely to be shopping/leisure destinations residing somewhat near the centre of their respective cities. This was cluster 2 and this would contain the optimal location to consider for a stakeholder growing their Indian restaurant business just from the variety of ammenities. This all points to the fact that these locations are venue dense and could help to consider which parts in Birmingham. Interesting picks from cluster 2:
- B10, B11, B28 are areas with several Indian restaurants. This insight could be followed up with a deep dive to see if you would benefit from locating near rivals. This idea is not recommended without further analysis.
- B4, B16 are at an excellent location with hardly any Indian restaurants to compete with at the nearest vicinity.

Cluster 4 is interesting because they're not areas that are 'venue dense' based on our used criteria. However there are Indian restaurants featured quite heavily in the data of this cluster. For instance we have B12 with 5 Indian restaurants within 1km radius (~36% of all venues returned). This suggests that the areas are likely to be saturated with the % of Indian restaurants present within the area. But there is a small chance that there's room for more and to really establish certain postcodes as a hub for Indian restaurants. I would imagine cluster 4 to not cost too much (assuming from the lack of nearby ammenities) and to therefore operate on a smaller budget. But this requires more of a deep dive and knowing more about stakeholder contraints.



## Conclusion <a name="conclusion"></a>

Purpose of this project was to identify B postcode areas close to centre with either hardly any Indian restaurants or with direct competitors in order to aid stakeholders in narrowing down the search for optimal location for a new branch from London. By calculating venue density distribution from Foursquare data we are able to see types of venues that are of interest to stakeholders and rank the top 5. Clustering of those locations was then performed in order to create major zones of interest (containing greatest number of potential locations) for final exploration by stakeholders.

Final decision on optimal restaurant location will be made by stakeholders based on specific characteristics locations in every recommended zone, taking into consideration additional factors like attractiveness of each location (proximity to park or water), levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood etc.